In [9]:
!pip install tensorflow==2.9.1
!pip install tf-encrypted

In [1]:
import tensorflow as tf

import tf_encrypted as tfe
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD
from tf_encrypted.player import DataOwner
from tf_encrypted.keras.datasets import LogisticArtificialDataset

config = tfe.get_config()

2024-01-30 16:25:08.988558: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 16:25:08.992316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-30 16:25:08.992329: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-30 16:25:10.697098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-30 16:25:10.697179: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [2]:
batch_size = 100
epochs = 20

In [6]:
train_dataset = LogisticArtificialDataset(batch_size=batch_size)
test_dataset = LogisticArtificialDataset(batch_size=batch_size, train=False)
train_client = DataOwner(config.get_player("train-client"), train_dataset.generator_builder())
test_client = DataOwner(config.get_player("test-client"), test_dataset.generator_builder())

train_iter = train_client.provide_data()
test_iter = test_client.provide_data()

In [7]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, train_dataset.num_features]))
model.add(tfe.keras.layers.Activation('sigmoid')) 

In [5]:
model.compile(optimizer=SGD(learning_rate=1), loss=BinaryCrossentropy())

# Train
model.fit(x=train_iter, epochs=epochs, steps_per_epoch=train_dataset.iterations)

# evaluate
model.evaluate(x=test_iter, metrics=["binary_accuracy"], steps=None)

Epoch 1/20
20/20 [==============================] - 12s 12ms/step - loss: 0.6921 - time: 0.6150
Epoch 2/20
20/20 [==============================] - 0s 11ms/step - loss: 0.6405 - time: 0.0080
Epoch 3/20
20/20 [==============================] - 0s 11ms/step - loss: 0.5974 - time: 0.0077
Epoch 4/20
20/20 [==============================] - 0s 11ms/step - loss: 0.5611 - time: 0.0081
Epoch 5/20
20/20 [==============================] - 0s 11ms/step - loss: 0.5301 - time: 0.0073
Epoch 6/20
20/20 [==============================] - 0s 11ms/step - loss: 0.5040 - time: 0.0068
Epoch 7/20
20/20 [==============================] - 0s 9ms/step - loss: 0.4816 - time: 0.0063
Epoch 8/20
20/20 [==============================] - 0s 10ms/step - loss: 0.4626 - time: 0.0067
Epoch 9/20
20/20 [==============================] - 0s 10ms/step - loss: 0.4459 - time: 0.0066
Epoch 10/20
20/20 [==============================] - 0s 9ms/step - loss: 0.4313 - time: 0.0063
Epoch 11/20
20/20 [==============================]

{'binary_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.96>}